# Random Oracle Model

## Random Oracles

A *random oracle* (RO) is an oracle that can be provided to an algorithm that takes inputs in $\{0, 1\}^*$ and returns elements from some set $S$.
The two key properties of a random oracle are: (1) it returns uniformly random and independent values from $S$ for distinct queries, and (2) it responds consistently, meaning identical queries always yield identical responses.

Formally, a random oracle can be described as follows.
Consider the random oracle $\mathcal{O}^H$ in the game below.
The oracle maintains a table $T$ to ensure consistency, where all entries are initially $\bot$ (undefined).
When queried with input $x$, it checks whether $T[x] = \bot$. If so, it samples a uniformly random element from $S$ and stores it as $T[x]$.
The oracle returns $T[x]$.

:::{note} Game: Random Oracle Example
:label: fig-ro-example

$$
\begin{array}{ll}
\underline{\text{Game } \mathsf{ROExample}:} & \underline{\text{Oracle } \mathcal{O}^H(x):} \\
T \leftarrow \emptyset & \text{if } T[x] = \bot \text{ then} \\
r \xleftarrow{\$} \{0,1\}^\lambda & \quad T[x] \xleftarrow{\$} S \\
h \leftarrow \mathcal{O}^H(r) & \text{return } T[x] \\
\text{return } \mathcal{A}^{\mathcal{O}^H}(h) &
\end{array}
$$
:::

Before examining the consequences of random oracles, we present a simple lemma that will be useful for analyzing games that differ only when certain events occur.

:::{important} Lemma: Difference Lemma
:label: lem-difference

Let $A$, $B$ and $E$ be events in a probability space. If $\Pr[A \wedge \neg E] = \Pr[B \wedge \neg E]$, then

$$
|\Pr[A] - \Pr[B]| \leq \Pr[E].
$$
:::

:::{dropdown} **Proof**

$$
\begin{aligned}
|\Pr[A] - \Pr[B]| &= |\Pr[A \wedge E] + \Pr[A \wedge \neg E] - (\Pr[B \wedge E] + \Pr[B \wedge \neg E])| \\
&= |\Pr[A \wedge E] - \Pr[B \wedge E]|
\end{aligned}
$$

Since $\Pr[A \wedge E] \leq \Pr[E]$ and $\Pr[B \wedge E] \leq \Pr[E]$, both terms are in $[0, \Pr[E]]$, so their difference is at most $\Pr[E]$.
:::

:::{important} Lemma
:label: lem-ro-indistinguishable

No adversary against $\mathsf{ROExample}$ making a polynomial number of queries can distinguish between being given $h = \mathcal{O}^H(r)$ for a random $r$ and being given a uniformly random value from $S$, except with negligible probability.
More precisely, let $\mathsf{Game~ROExample}$ be as defined in [](#fig-ro-example).
For any adversary $\mathcal{A}$ making $q$ queries to the random oracle $\mathcal{O}^H$, we have

$$
\left|\Pr[\mathsf{ROExample} = 1] - \Pr[\mathsf{ROExample}_2 = 1]\right| \leq \frac{q}{2^\lambda}
$$

where $\mathsf{ROExample}_2$ is defined in [](#fig-ro-game-hop).
:::

:::{tip} Remark

Unlike the security games we have seen so far where there are winning and losing conditions, $\mathsf{ROExample}$ simply runs the adversary and returns whatever the adversary outputs.
[](#lem-ro-indistinguishable) simply states that the adversary's behavior (its output distribution) is identical in both games with probability at least $1 - q/2^\lambda$.
:::

:::{note} Game: Game hop for random oracle indistinguishability
:label: fig-ro-game-hop

$$
\begin{array}{ll}
\underline{\text{Game } \mathsf{ROExample}_2:} & \underline{\text{Oracle } \mathcal{O}^H(x):} \\
T \leftarrow \emptyset & \colorbox{yellow}{$\text{assert } x \neq r$} \\
r \xleftarrow{\$} \{0,1\}^\lambda & \text{if } T[x] = \bot \text{ then} \\
\colorbox{yellow}{$\phantom{h \leftarrow \mathcal{O}^H(r)}$} & \quad T[x] \xleftarrow{\$} S \\
\colorbox{yellow}{$h' \xleftarrow{\$} S$} & \text{return } T[x] \\
\text{return } \mathcal{A}^{\mathcal{O}^H}(\colorbox{yellow}{$h'$}) &
\end{array}
$$
:::

:::{dropdown} **Proof**

By inspection of the code, we observe that $\mathsf{ROExample}_2$ is identical to $\mathsf{ROExample}$ except that:

1. The input to $\mathcal{A}$ is changed from $h = \mathcal{O}^H(r)$ to a uniformly random $h' \xleftarrow{\$} S$.
2. The oracle $\mathcal{O}^H$ asserts that no query equals $r$.

Since $h'$ is uniformly random in $S$ and has the same distribution as $\mathcal{O}^H(r)$ (when $r$ is not queried), the games are identical unless the adversary queries $r$ to the oracle.

We apply the Difference Lemma ([](#lem-difference)).
Let $E$ denote the event that $\mathcal{A}$ queries $r$ to the oracle.

When $\neg E$ occurs (i.e., $r$ is not queried), the two games behave identically, so $\Pr[\mathsf{ROExample} = 1 \wedge \neg E] = \Pr[\mathsf{ROExample}_2 = 1 \wedge \neg E]$.

By [](#lem-difference):

$$
\begin{aligned}
\left|\Pr[\mathsf{ROExample} = 1] - \Pr[\mathsf{ROExample}_2 = 1]\right| &\leq \Pr[E] \\
&= \Pr[\exists i \in [q] : x_i = r] \\
&\leq \sum_{i=1}^q \Pr[x_i = r] \quad \text{(union bound)} \\
&= \sum_{i=1}^q \frac{1}{2^\lambda} = \frac{q}{2^\lambda}
\end{aligned}
$$

where $x_1, \ldots, x_q$ are the queries made by $\mathcal{A}$ to $\mathcal{O}^H$.
:::

## ROM

For a large class of cryptographic schemes, the properties of hash functions seen so far are insufficient to prove security.
Instead, we prove security of a modified scheme where hash function evaluations are replaced with random oracle calls.
This is the *random oracle model* (ROM).
Random oracles cannot be instantiated in the real world because their output must be distributed uniformly at random unless explicitly queried.
On the other hand, the output of a hash function is deterministic once the key is known.
Moreover, there's no way to formally define an "explicit query" for public hash functions.

There exist contrived schemes provably secure in the ROM that become insecure when instantiated with any concrete hash function (see optional exercise).
Despite these theoretical concerns, the ROM has proven very useful in practice for analyzing real-world cryptographic protocols.

## Hash Commitments

:::{note} Definition: Hash Commitment Scheme
:label: def-hash-commitment

Let $\mathsf{H}$ be a hash function.
The hash commitment scheme $\mathsf{HashCom}[\mathsf{H}]$ is defined as follows:

- $\mathsf{Setup}(1^\lambda) \rightarrow \mathsf{params}$: Output $\mathsf{params} = (\mathcal{M} \leftarrow \{0, 1\}^*, \mathcal{R} \leftarrow \{0, 1\}^\lambda)$.
- $\mathsf{Commit}(\mathsf{params}, m, r) \rightarrow C$: Output $C = \mathsf{H.Eval}(m \| r)$.
:::

:::{important} Lemma
:label: lem-hash-com-hiding

The hash commitment scheme $\mathsf{HashCom}[\mathsf{H}]$ is hiding in the random oracle model for $\mathsf{H}$.
More precisely, for any algorithm $\mathcal{A}$ making at most $q$ queries to $\mathsf{H}$,

$$
\mathsf{Adv}^{\mathsf{Hide}}_{\mathcal{A}, \mathsf{HashCom}} \leq \frac{q}{2^\lambda}.
$$
:::

The proof is left as an exercise (see [](#ex-hash-com-hiding)).

## Taproot Commitments

:::{note} Definition: Taproot Commitment Scheme
:label: def-taproot-commitment

Let $\mathsf{GrGen}$ be a group generation algorithm and $\mathsf{H}$ be a hash function with output in $\mathbb{Z}_p$ where $p$ is the group order.
The Taproot commitment scheme $\mathsf{TapCom}[\mathsf{GrGen}, \mathsf{H}]$ is defined as follows:

- $\mathsf{Setup}(1^\lambda) \rightarrow \mathsf{params}$: Run $\mathsf{gparam} \leftarrow \mathsf{GrGen}(1^\lambda)$ and $\kappa \leftarrow \mathsf{H.Gen}(1^\lambda)$. Output $\mathsf{params} = (\mathsf{gparam}, \kappa, \mathcal{M} \leftarrow \{0,1\}^*, \mathcal{R} \leftarrow \mathbb{G})$.
- $\mathsf{Commit}(\mathsf{params}, m, R) \rightarrow C$: Parse $\mathsf{params}$ to obtain $g$ and $\kappa$. Output $C = R \cdot g^{\mathsf{H.Eval}(\kappa, (R, m))}$.
:::

:::{important} Lemma
:label: lem-taproot-binding

Let $\mathsf{GrGen}$ be a group generation algorithm and $\mathsf{H}$ be a hash function with output in $\mathbb{Z}_p$ where $p$ is the group order.
The Taproot commitment scheme $\mathsf{TapCom}[\mathsf{GrGen}, \mathsf{H}]$ is binding in the random oracle model for $\mathsf{H}$.
More precisely, for any algorithm $\mathcal{A}$ making at most $q$ queries to $\mathsf{H}$,

$$
\mathsf{Adv}^{\mathsf{Bind}}_{\mathcal{A}, \mathsf{TapCom}} \leq \frac{(q+2)^2}{2^\lambda}.
$$
:::

The proof is left as an exercise (see [](#ex-taproot-binding)).

## Exercises

:::{exercise}
:label: ex-rom-contrived-hashcom

Consider a modified hash commitment scheme $\mathsf{HashCom}_y[\mathsf{H}]$ defined as follows:

- $\mathsf{Setup}(1^\lambda) \rightarrow \mathsf{params}$: Same as $\mathsf{HashCom}[\mathsf{H}]$.
- $\mathsf{Commit}(\mathsf{params}, m, r) \rightarrow C$:

$$
C = \begin{cases}
\mathsf{H.Eval}(m) & \text{if } \mathsf{H.Eval}(0) = y \\
\mathsf{H.Eval}(m \| r) & \text{otherwise}
\end{cases}
$$

1. Using [](#lem-hash-com-hiding), argue that for all $y \in \{0,1\}^\lambda$, $\mathsf{HashCom}_y[\mathsf{H}]$ is hiding in the ROM.
2. Show that there exists $y$ such that $\mathsf{HashCom}_y$ instantiated with SHA256 is not hiding.
:::

:::{dropdown} **Solution**

1. In the ROM, $\mathsf{H}$ behaves as a random oracle. For any fixed $y$, we have $\Pr[\mathsf{H.Eval}(0) = y] = \frac{1}{2^\lambda}$, which is negligible. With overwhelming probability, the scheme behaves identically to the standard $\mathsf{HashCom}[\mathsf{H}]$, which is hiding by [](#lem-hash-com-hiding).

   More formally, the hiding advantage is bounded by:

   $$
   \mathsf{Adv}^{\mathsf{Hide}}_{\mathcal{A}, \mathsf{HashCom}_y} \leq \Pr[\mathsf{H.Eval}(0) = y] + \mathsf{Adv}^{\mathsf{Hide}}_{\mathcal{A}, \mathsf{HashCom}} \leq \frac{1}{2^\lambda} + \frac{2q}{2^\lambda}
   $$

2. Let $y = \text{SHA256}(0)$. When instantiated with SHA256, we always have $\mathsf{H.Eval}(0) = y$, so the commitment becomes $C = \text{SHA256}(m)$ without any randomness. This is deterministic and therefore not hiding: given two messages $m_0, m_1$, the adversary can compute $\text{SHA256}(m_0)$ and $\text{SHA256}(m_1)$ and compare with $C$ to determine which message was committed to.
:::

:::{exercise}
:label: ex-hash-com-hiding

Prove [](#lem-hash-com-hiding) using a similar technique as in [](#lem-ro-indistinguishable).
:::

:::{dropdown} **Solution**

The hiding game for $\mathsf{HashCom}[\mathsf{H}]$ where $\mathsf{H}$ is modeled as a random oracle is shown below.

**Game: The hiding game for $\mathsf{HashCom}[\mathsf{H}]$ in the random oracle model**

$$
\begin{array}{ll}
\underline{\text{Game } \mathsf{ComHide}^{\mathsf{HashCom}}:} & \underline{\text{Oracle } \mathcal{O}^H(x):} \\
\mathsf{params} \leftarrow \mathsf{Setup}(1^\lambda) & \text{if } T[x] = \bot \text{ then} \\
(m_0, m_1) \leftarrow \mathcal{A}^{\mathcal{O}^H}(\mathsf{params}) & \quad T[x] \xleftarrow{\$} \{0,1\}^\lambda \\
T \leftarrow \emptyset & \text{return } T[x] \\
b \xleftarrow{\$} \{0,1\} & \\
r \xleftarrow{\$} \{0,1\}^\lambda & \\
c \leftarrow \mathcal{O}^H(m_b \| r) & \\
b' \leftarrow \mathcal{A}^{\mathcal{O}^H}(c) & \\
\text{return } b = b' &
\end{array}
$$

Now consider the modified game:

**Game: Modified hiding game for $\mathsf{HashCom}[\mathsf{H}]$ with changes highlighted**

$$
\begin{array}{ll}
\underline{\text{Game } \mathsf{ComHide}_2^{\mathsf{HashCom}}:} & \underline{\text{Oracle } \mathcal{O}^H(x):} \\
\mathsf{params} \leftarrow \mathsf{Setup}(1^\lambda) & \text{if } T[x] = \bot \text{ then} \\
(m_0, m_1) \leftarrow \mathcal{A}(\mathsf{params}) & \quad T[x] \xleftarrow{\$} \{0,1\}^\lambda \\
T \leftarrow \emptyset & \text{return } T[x] \\
b \xleftarrow{\$} \{0,1\} & \\
r \xleftarrow{\$} \{0,1\}^\lambda & \\
\colorbox{yellow}{$\phantom{c \leftarrow \mathcal{O}^H(m_b \| r)}$} & \\
\colorbox{yellow}{$c' \xleftarrow{\$} \{0,1\}^\lambda$} & \\
b' \leftarrow \mathcal{A}^{\mathcal{O}^H}(\colorbox{yellow}{$c'$}) & \\
\text{return } b = b' &
\end{array}
$$

Note that in $\mathsf{ComHide}_2^{\mathsf{HashCom}}$, the value $c'$ is uniformly random and independent of $b$, so the adversary's view is independent of $b$.
Therefore:

$$
\Pr[\mathsf{ComHide}_2^{\mathsf{HashCom}} = 1] = \frac{1}{2}
$$

The games differ only when $\mathcal{A}$ queries some value to the oracle that ends with $r$.
This is because in $\mathsf{ComHide}^{\mathsf{HashCom}}$, the challenger queries $\mathcal{O}^H(m_b \| r)$ to compute the commitment, while in $\mathsf{ComHide}_2^{\mathsf{HashCom}}$, it samples $c'$ uniformly at random without querying the oracle.
If the adversary never queries a value ending with $r$, then both games provide the adversary with a uniformly random commitment value and behave identically.
However, if the adversary does query $m_b \| r$ (or any other value ending with $r$), the games diverge: in $\mathsf{ComHide}^{\mathsf{HashCom}}$, the oracle returns the value consistent with $c$, while in $\mathsf{ComHide}_2^{\mathsf{HashCom}}$, the oracle returns a fresh random value.
By the Difference Lemma ([](#lem-difference)), we can bound the difference between the games by the probability of this event:

$$
\begin{aligned}
\mathsf{Adv}^{\mathsf{Hide}}_{\mathcal{A}, \mathsf{HashCom}} &= \left|\Pr[\mathsf{ComHide}^{\mathsf{HashCom}} = 1] - \frac{1}{2}\right| \\
&= \left|\Pr[\mathsf{ComHide}^{\mathsf{HashCom}} = 1] - \Pr[\mathsf{ComHide}_2^{\mathsf{HashCom}} = 1]\right| \\
&\leq \Pr[\exists i \in [q] : x_i \text{ ends with } r]
\end{aligned}
$$

Since $r$ is uniformly random from $\{0,1\}^\lambda$ and unknown to $\mathcal{A}$, for any query $x_i$ chosen by the adversary, the probability that its last $\lambda$ bits equal $r$ is at most $1/2^\lambda$.
By the union bound over all $q$ queries:

$$
\begin{aligned}
\Pr[\exists i \in [q] : x_i \text{ ends with } r] &\leq \sum_{i=1}^q \Pr[x_i \text{ ends with } r] \\
&\leq \sum_{i=1}^q \frac{1}{2^\lambda} = \frac{q}{2^\lambda}
\end{aligned}
$$
:::

:::{exercise}
:label: ex-taproot-binding

Prove [](#lem-taproot-binding).
:::

:::{dropdown} **Solution**

Let $\mathcal{A}$ be an adversary against the binding property of $\mathsf{TapCom}[\mathsf{GrGen}, \mathsf{H}]$. The adversary wins if it outputs $(m_0, m_1, R_0, R_1)$ with $m_0 \neq m_1$ such that:

$$
R_0 \cdot g^{\mathsf{H}(R_0, m_0)} = R_1 \cdot g^{\mathsf{H}(R_1, m_1)}
$$

Rearranging, this equality holds if and only if:

$$
R_0 \cdot g^{\mathsf{H}(R_0, m_0)} \cdot R_1^{-1} = g^{\mathsf{H}(R_1, m_1)}
$$

Our strategy is to check for binding collisions as queries are made to the random oracle.
Consider the distinct queries to the random oracle of the form $(R_i, m_i)$ and define the collision predicate:

$$
c(i, j) = \text{true} \iff R_i \cdot g^{\mathsf{H}(R_i, m_i)} \cdot R_j^{-1} = g^{\mathsf{H}(R_j, m_j)}
$$

The probability that the adversary breaks binding is:

$$
\mathsf{Adv}^{\mathsf{Bind}}_{\mathcal{A}, \mathsf{TapCom}} = \Pr\left[\bigvee_{i=1}^{q'} \bigvee_{j=i+1}^{q'} c(i,j) = \text{true}\right]
$$

where $q'$ is the number of distinct queries.

For all $j > i$, we have $\Pr[c(i,j) = \text{true}] = \frac{1}{p}$, since $\mathsf{H}(R_j, m_j)$ is a fresh uniformly random value in $\mathbb{Z}_p$, independent of $R_i$, $\mathsf{H}(R_i, m_i)$, and $R_j$.

By the union bound:

$$
\begin{aligned}
\mathsf{Adv}^{\mathsf{Bind}}_{\mathcal{A}, \mathsf{TapCom}} &= \Pr\left[\bigvee_{i=1}^{q'} \bigvee_{j=i+1}^{q'} c(i,j) = \text{true}\right] \\
&\leq \sum_{i=1}^{q'} \sum_{j=i+1}^{q'} \Pr[c(i,j) = \text{true}] \\
&= \sum_{i=1}^{q'} \sum_{j=i+1}^{q'} \frac{1}{p} \\
&= \binom{q'}{2} \cdot \frac{1}{p} < \frac{(q')^2}{2p}
\end{aligned}
$$

The total number of queries includes the adversary's queries (at most $q$) plus the 2 queries made by the binding game when checking $C_0 = C_1$. Thus $q' \leq q + 2$.

Since $p \geq 2^{\lambda-1}$, we have:

$$
\mathsf{Adv}^{\mathsf{Bind}}_{\mathcal{A}, \mathsf{TapCom}} < \frac{(q+2)^2}{2p} \leq \frac{(q+2)^2}{2 \cdot 2^{\lambda-1}} = \frac{(q+2)^2}{2^\lambda}
$$
:::

:::{exercise}
:label: ex-ro-output-distribution

**Optional:**

1. Let $\mathsf{H}$ be a hash function with output in $\{0,1\}^\lambda$ and let $\kappa$ be chosen uniformly at random from a set of size $2^\lambda$. Define $A = \mathsf{H.Eval}(\kappa, 0) \| \mathsf{H.Eval}(\kappa, 1)$. What is the size of the set over which $A$ is uniformly distributed?
2. Let $\mathsf{H}$ be a random oracle with output in $\{0,1\}^\lambda$. Define $B = \mathsf{H}(0) \| \mathsf{H}(1)$. What is the size of the set over which $B$ is uniformly distributed?
:::

:::{dropdown} **Solution**

1. There are $2^\lambda$ possibilities for $\kappa$, and each determines a unique value of $A$. Therefore, $A$ is uniformly distributed over a set of size at most $2^\lambda$.
2. Since $\mathsf{H}(0)$ and $\mathsf{H}(1)$ are independent uniformly random values in $\{0,1\}^\lambda$, $B$ is uniformly distributed over a set of size $2^{2\lambda}$.
:::

:::{exercise}
:label: ex-xor-preimage

**Optional:** Let $\mathsf{H}$ be a random oracle with output in $\{0,1\}^\lambda$. Define the function $G: (\{0,1\}^*)^{\lambda+1} \rightarrow \{0,1\}^\lambda$ as:

$$
G(x_0, x_1, \ldots, x_\lambda) = \mathsf{H}(x_0) \oplus \mathsf{H}(x_1) \oplus \cdots \oplus \mathsf{H}(x_\lambda)
$$

where $\oplus$ denotes bitwise XOR. Is $G$ preimage resistant? That is, given $y \in \{0,1\}^\lambda$, is it hard for a PPT adversary with oracle access to $\mathsf{H}$ to find $(x_0, \ldots, x_\lambda)$ such that $G(x_0, \ldots, x_\lambda) = y$?
:::

:::{dropdown} **Solution**

No, $G$ is not preimage resistant. Wagner's algorithm for the generalized birthday problem can find a preimage efficiently.
:::

:::{exercise}
:label: ex-rom-quantum

**Optional:** Why is the ROM not considered adequate to model quantum computation?
:::

:::{exercise}
:label: ex-rom-diagonalization

**Optional:** Study the diagonalization argument in Section 4 of Canetti, Goldreich, and Halevi (STOC 1998).
Explain how this argument demonstrates the existence of a contrived signature scheme that is provably secure in the random oracle model but becomes insecure when the random oracle is instantiated with any concrete hash function (not just a specific fixed hash function as considered in [](#ex-rom-contrived-hashcom)).
:::